In [ ]:
!pip install jiwer
!pip install gradio
!pip install hopsworks
!pip install python-dotenv

In [2]:
# %load 2_feature_pipeline.py
import sys
import pandas as pd
from datetime import datetime
import time 
import os
import requests
import json
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/ID2223-project/air quality prediction2')
from functions import *


Mounted at /content/drive


In [3]:
date_today = datetime.now().strftime("%Y-%m-%d")

In [4]:
cities = ['Guangzhou']
data_air_quality = [get_air_quality_data(city) for city in cities]
data_weather = [get_weather_data(city, date_today) for city in cities]

In [5]:
df_air_quality = get_air_quality_df(data_air_quality)
df_air_quality

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,pm10_avg,pm10_max,pm10_min,pm25,pm25_max,pm25_min
0,Guangzhou,45,1673654400000,85.5,1008,45,23,1,1,1,58,58,58,157,157,157


In [6]:
df_air_quality = df_air_quality.drop(labels=['city','aqi','iaqi_h','iaqi_p','iaqi_pm10','iaqi_t','o3_avg','o3_max','o3_min','pm10_avg','pm10_max','pm10_min','pm25_max','pm25_min'], axis=1)
# check the csv function
df_air_quality

,date,pm25
0,1673654400000,157


In [7]:
df_weather = get_weather_df(data_weather)
df_weather

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Guangzhou,1673654400000,28.0,18.0,23.0,30.0,18.0,23.3,19.7,83.6,...,40.0,20.9,208.2,1008.8,71.9,11.4,192.2,16.7,7.0,"Rain, Partially cloudy"


In [8]:
df_weather = df_weather.drop(labels=['feelslikemax','temp','precipprob','windspeed','cloudcover','precip','tempmax','uvindex','solarradiation','solarenergy','winddir','snow','snowdepth','conditions','city'], axis=1)

In [9]:
df_weather

,date,tempmin,feelslikemin,feelslike,dew,humidity,precipcover,windgust,sealevelpressure,visibility
0,1673654400000,18.0,18.0,23.3,19.7,83.6,16.67,40.0,1008.8,11.4


In [10]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store() 

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5465
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg_clean',
    version = 1
)

air_quality_fg.insert(df_air_quality)
weather_fg.insert(df_weather)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5465/jobs/named/air_quality_fg_1_offline_fg_backfill/executions


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5465/jobs/named/weather_fg_clean_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f33cd631b80>, None)